In [1]:
import pandas as pd

from tensorflow.keras.models import Sequential;
from tensorflow.keras.layers import Dense, Dropout;
from tensorflow.keras.callbacks import EarlyStopping;

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
plt.style.use('seaborn')

In [3]:
def get_data(row_limit, from_date_customer, from_date_dispute):
    if row_limit > 0:
        row_limit = 'limit ' + str(row_limit)
    else:
        row_limit = ''

    query = """SELECT DISTINCT
            CUSTOMER_NO_ANON,
            COUNT(CUSTOMER_NO_ANON) OVER (PARTITION BY CUSTOMER_NO_ANON, OFFER_DESC) Months_With_Offer,
            CASE
                WHEN CUSTOMER_TYPE_DESC = 'Consumer' THEN 'Consumer'
                WHEN CUSTOMER_TYPE_DESC = 'Business' THEN 'Business'
            ELSE
                'Other'
            END as CUSTOMER_TYPE_DESC,
            OFFER_DESC,
            COMMITMENT_PERIOD,
            TOTAL_AMOUNT,
            AVG(TOTAL_AMOUNT) OVER (Partition by CUSTOMER_NO_ANON, OFFER_DESC, PRIM_RESOURCE_VAL_ANON) Avg_Amount,
            STDDEV_POP(TOTAL_AMOUNT) OVER (Partition by CUSTOMER_NO_ANON, OFFER_DESC, PRIM_RESOURCE_VAL_ANON) Std_Amount,
            CASE
                WHEN LOWER(CREDIT_CLASS_DESC) = 'spclow' THEN 'special low'
                WHEN LOWER(CREDIT_CLASS_DESC) = 'highrisk' THEN 'high'
                WHEN LOWER(CREDIT_CLASS_DESC) = 'medrisk' THEN 'medium'
                WHEN LOWER(CREDIT_CLASS_DESC) = 'newrisk' THEN 'new'
                WHEN LOWER(CREDIT_CLASS_DESC) = 'lowrisk' THEN 'low'
            ELSE
                LOWER(CREDIT_CLASS_DESC)
            END as CREDIT_CLASS_DESC,
            SERVICE_TYPE,
            BILL_MONTH,
             IF(TIMESTAMP_DIFF(Disputes.RESOLUTION_DATE, BILL_MONTH, DAY) < 122 AND TIMESTAMP_DIFF(Disputes.RESOLUTION_DATE, BILL_MONTH, DAY) >=0, 1, 0) Within_Dispute_Period
            FROM `bcx-insights.telkom_customerexperience.customerdata_20190902_00_anon` Customers
            LEFT JOIN
            (SELECT DISTINCT ACCOUNT_NO_ANON, RESOLUTION_DATE FROM `bcx-insights.telkom_customerexperience.disputes_20190903_00_anon`
            WHERE STATUS_DESC = 'Justified' and RESOLUTION_DATE >= '{0}') Disputes
            on Disputes.ACCOUNT_NO_ANON = Customers.CUSTOMER_NO_ANON
            WHERE CUSTOMER_TYPE_DESC <> 'Government' AND
            BILL_MONTH >= '{1}'
            {2}""".format(from_date_customer, from_date_dispute, row_limit)

    df = pd.io.gbq.read_gbq(query, project_id='bcx-insights', dialect='standard')
    
    return df

In [4]:
def preprocess(df):
    columns_to_drop = ['CUSTOMER_NO_ANON', 'BILL_MONTH']
    df = df.drop(columns_to_drop, 1)
    df['COMMITMENT_PERIOD'] = df['COMMITMENT_PERIOD'].astype(str)  
    
    types = df.dtypes
    
    numeric_cols = list(types[types==float].index)    
    categorical_cols = list(types[types!=float].index)
    
    ss = StandardScaler()    
    scale_cols = pd.DataFrame(ss.fit_transform(df[numeric_cols]), columns=numeric_cols, index=df.index)
        
    df = df[categorical_cols].join(scale_cols)    
    df = pd.get_dummies(df)
    df = df.sort_index(1)
        
    return df

In [5]:
def upsample(data, repetitions):
    positive_samples = data[data['Within_Dispute_Period'] == 1]
    for _ in range(repetitions):
        data = data.append(positive_samples)
    return data.reset_index(drop=True)

In [6]:
data_limit = 2000000
df = get_data(data_limit, '2019-04-01', '2019-01-01')

In [7]:
df.head()

,CUSTOMER_NO_ANON,Months_With_Offer,CUSTOMER_TYPE_DESC,OFFER_DESC,COMMITMENT_PERIOD,TOTAL_AMOUNT,Avg_Amount,Std_Amount,CREDIT_CLASS_DESC,SERVICE_TYPE,BILL_MONTH,RESOLUTION_DATE,Within_Dispute_Period
0,8434707970976031174,23,Business,FreeMe Unlimited Deal,24,1610.26,1408.977500,532.543438,new,MOBILE,2019-06-01 00:00:00+00:00,NaT,0
1,8435805989213008025,6,Other,GC Unlimited Broadband 10Mbps,24,262.41,274.905000,27.939669,new,FIXED,2019-06-01 00:00:00+00:00,NaT,0
2,8440657939074637625,8,Other,Unlimited Home DSL 4Mbps (24 month),24,568.97,627.672500,73.496002,low,FIXED,2019-03-01 00:00:00+00:00,NaT,0
3,8446369958128414835,8,Other,Unlimited Business DSL (8 Mbps) 24 Months,24,533.88,769.421250,435.180923,special low,FIXED,2019-03-01 00:00:00+00:00,NaT,0
4,8448380172178352586,3,Consumer,SmartInternet Top-Up 1GB Deal,24,60.00,67.096667,10.036202,new,MOBILE,2019-06-01 00:00:00+00:00,NaT,0


In [8]:
dispute_prop = round(df['Within_Dispute_Period'].sum()/df['Within_Dispute_Period'].count(), 3)
print('Proportion with disputed:', dispute_prop)

Proportion with disputed: 0.005


In [9]:
df = upsample(df, int(0.3/dispute_prop))
accounts = df['CUSTOMER_NO_ANON']
X = preprocess(df.drop('Within_Dispute_Period', 1))
y = df['Within_Dispute_Period']

df = None

In [10]:
with open('columns.txt', 'w') as f:
    for c in X.columns:
        f.write(c + '\n')

In [11]:
print('Proportion with disputed:', round(y.sum()/y.count(), 3))
print('Number of samples:', len(X))

Proportion with disputed: 0.224
Number of samples: 2566100


In [12]:
X.head()

,Avg_Amount,COMMITMENT_PERIOD_0,COMMITMENT_PERIOD_12,COMMITMENT_PERIOD_18,COMMITMENT_PERIOD_24,COMMITMENT_PERIOD_3,COMMITMENT_PERIOD_36,COMMITMENT_PERIOD_48,COMMITMENT_PERIOD_6,COMMITMENT_PERIOD_60,...,OFFER_DESC_i-VPN ATM Subrate,SERVICE_TYPE_-,SERVICE_TYPE_BROADBAND,SERVICE_TYPE_FIXED,SERVICE_TYPE_INTERNET,SERVICE_TYPE_MOBILE,SERVICE_TYPE_VAS,SERVICE_TYPE_VOICE,Std_Amount,TOTAL_AMOUNT
0,0.046816,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.038236,0.065388
1,-0.008600,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,-0.013762,-0.005778
2,0.008638,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,-0.009067,0.010408
3,0.015564,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0.028203,0.008555
4,-0.018755,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,-0.015607,-0.016466


In [13]:
print(X.shape, y.shape)

(2566100, 1017) (2566100,)


In [14]:
print('Number of features:', len(X.columns))

Number of features: 1017


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [16]:
print('Proportion with disputed (test):', round(y_test.sum()/y_test.count(), 3))

Proportion with disputed (test): 0.224


In [46]:
model = Sequential()

model.add(Dense(8, input_dim=X.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


es = EarlyStopping(monitor='val_loss', patience=15)


model.compile(optimizer='adam',
              loss='binary_crossentropy',          
              metrics=['accuracy']);

print(model.summary())

history = model.fit(X_train, y_train, 
                    validation_data=[X_test, y_test], 
                    epochs=40, callbacks=[es],
                   batch_size=data_limit//5)
                   

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 8144      
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 5         
Total params: 8,185
Trainable params: 8,185
Non-trainable params: 0
_________________________________________________________________
None
Train on 1796270 samples, validate on 769830 samples
Epoch 1/40
1796270/1796270 [==============================] - 117s 65us/sample - loss: 2.4841 - acc: 0.2266 - val_loss: 2.3014 - val_acc: 0.2297
Epoch 2/40
1796270/1796270 [==============================] - 116s 64us/sample - loss: 2.1654 - acc: 0.2307 - val_loss: 1.9807 - val_acc: 0.2350
Epoch 3/40
1796270/1796270 [======

KeyboardInterrupt: 

In [ ]:
model.save('dispute_model.h5')

In [ ]:
y_pred = (model.predict(X_test) > 0.5) * 1

In [ ]:
X.head()

In [ ]:
confusion = pd.DataFrame(confusion_matrix(y_test, y_pred, labels=[0, 1]), columns=['no dispute', 'dispute'], index=['no dispute', 'dispute'])

In [ ]:
confusion

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
plt.plot(history.history['val_acc']);

In [ ]:
dataset = pd.DataFrame(model.predict(X_test), columns=['probability'], index=X_test.index)
dataset = dataset[dataset['probability'] >= 0.3]
dataset['category'] = dataset['probability'].apply(lambda x: 'green' if x <= 0.5 else 'orange' if x < 0.75 else 'red')
dataset = dataset.join(accounts)
dataset = dataset.groupby('CUSTOMER_NO_ANON', as_index=False).max()

In [ ]:
dataset['bin'] = pd.cut(dataset['probability'], bins=15).apply(lambda x: str(x.right) if x.left > dataset['probability'].min() else '{0} - {1}'.format(x.left, x.right))
dataset = dataset.sort_values('bin')

print(dataset.head())

In [ ]:
dataset = dataset[['bin', 'CUSTOMER_NO_ANON', 'category']].groupby(['bin', 'category'], as_index=False).count()

In [ ]:
for c in ['green', 'orange', 'red']:
    data = dataset[dataset['category'] == c]
    plt.bar(data['bin'], data['CUSTOMER_NO_ANON'], color=c);
plt.show()